# Recommender Systems in Python: Beginner Tutorial

From here: https://www.datacamp.com/community/tutorials/recommender-systems-python

a simplified clone of IMDB Top 250 Movies using metadata collected from IMDB.

The following are the steps involved:

Decide on the metric or score to rate movies on.
Calculate the score for every movie.
Sort the movies based on the score and output the top results.

its weighted rating formula as your metric/score. Mathematically, it is represented as follows:

Weighted Rating (WR) = (v/(v+m)*R)+(m/(v+m)*C)
where,

v is the number of votes for the movie;
m is the minimum votes required to be listed in the chart;
R is the average rating of the movie; And
C is the mean vote across the whole report
You already have the values to v (vote_count) and R (vote_average) for each movie in the dataset. It is also possible to directly calculate C from this data.

What you need to determine is an appropriate value for m, the minimum votes required to be listed in the chart. There is no right value for m. You can view it as a preliminary negative filter that ignores movies which have less than a certain number of votes. The selectivity of your filter is up to your discretion.

In this case, you will use the 90th percentile as your cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least 90% of the movies in the list. (On the other hand, if you had chosen the 75th percentile, you would have considered the top 25% of the movies in terms of the number of votes garnered. As the percentile decreases, the number of movies considered increases. Feel free to play with this value and observe the changes in your final chart).

In [1]:
# Import Pandas
import pandas as pd

# Load Movies Metadata
metadata = pd.read_csv('movies_metadata.csv', low_memory=False)

# Print the first three rows
metadata.head(3)

adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497                en   
2                                   NaN  15602  tt0113228                en   

     original_title                                           overview  \
0         Toy Story  Led by Woody, Andy's toys live happily in his ...   
1           Jumanji  When siblings Judy and Peter discover an encha...   
2  Grumpier Old Men  A family wedding reignites the ancient feud be...   

     ...     release_date      revenue runtime  \
0    ...       1995-10-30  373554033.0    81.0   
1    ...       1995-12-15  262797249.0   104.0   
2    ...       1995-12-22          0.0   101.0   

                                    spoken_languages    status  \
0           [{'iso_639_1': 'en', 'name': 'English'}]  Released   
1  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...  Released   
2           [{'iso_639_1': 'en', 'name': 'English'}]  Released   

                                             tagline             title  video  \
0                                                NaN         Toy Story  False   
1          Roll the dice and unleash the excitement!           Jumanji  False   
2  Still Yelling. Still Fighting. Still Ready for...  Grumpier Old Men  False   

  vote_average vote_count  
0          7.7     5415.0  
1          6.9     2413.0  
2          6.5       92.0  

[3 rows x 24 columns]

In [2]:
# Calculate the vote average cross all movies.
C = metadata['vote_average'].mean()
print(C)


5.61820721513


In [3]:
# Calculate the minimum number of votes required to be in the chart, m
m = metadata['vote_count'].quantile(0.90)
print(m)
# Filter out all qualified movies into a new DataFrame
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

160.0


(4555, 24)

In [4]:
# Function that computes the weighted rating of each movie. Weighted Rating (WR) = (v/(v+m)*R)+(m/(v+m)*C)
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [5]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [6]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(15)

title  vote_count  vote_average     score
314           The Shawshank Redemption      8358.0           8.5  8.445869
834                      The Godfather      6024.0           8.5  8.425439
10309      Dilwale Dulhania Le Jayenge       661.0           9.1  8.421453
12481                  The Dark Knight     12269.0           8.3  8.265477
2843                        Fight Club      9678.0           8.3  8.256385
292                       Pulp Fiction      8670.0           8.3  8.251406
522                   Schindler's List      4436.0           8.3  8.206639
23673                         Whiplash      4376.0           8.3  8.205404
5481                     Spirited Away      3968.0           8.3  8.196055
2211                 Life Is Beautiful      3643.0           8.3  8.187171
1178            The Godfather: Part II      3418.0           8.3  8.180076
1152   One Flew Over the Cuckoo's Nest      3001.0           8.3  8.164256
351                       Forrest Gump      8147.0           8.2  8.150272
1154           The Empire Strikes Back      5998.0           8.2  8.132919
1176                            Psycho      2405.0           8.3  8.132715

## Now, let do the Content-Based Recommender

This methods recommends movies that are similar to a particular movie. More specifically, it computes pairwise similarity scores for all movies based on their plot descriptions and recommends movies based on that similarity score.

The plot description is availablein the metadata dataset as the 'overview' feature. 

To compute the similarity, compute the word vectors of each overview or document. The Term Frequency-Inverse Document Frequency (TF-IDF) vectors for each document will be computed. This will give  a matrix where each column represents a word in the overview vocabulary (all the words that appear in at least one document) and each row represents a movie, as before.

In its essence, the TF-IDF score is the frequency of a word occurring in a document, down-weighted by the number of documents in which it occurs. This is done to reduce the importance of words that occur frequently in plot overviews and therefore, their significance in computing the final similarity score.

Then, similarity score can be computed on the TF-IDF matrix uisng variaty methods, such as the euclidean, the Pearson and the cosine similarity scores. Here cosine similarity score will be used since it is independent of magnitude and is relatively easy and fast to calculate (especially when used in conjunction with TF-IDF scores, which will be explained later)

In [7]:
#Print plot overviews of the first 5 movies.
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [8]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata['overview'] = metadata['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(45466, 75823)

In [9]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [10]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

In [11]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

In [12]:
get_recommendations('The Truman Show')

24535       The Prodigy
28568      Chronic Town
25476          Top Five
5203          Delirious
4500        Next of Kin
15601     Call Me Madam
10390            Capote
749      The Grass Harp
18348       Local Color
20985        The Twonky
Name: title, dtype: object

## Improve the recommender based on more information
Credits, Genres and Keywords Based Recommender


In [13]:
# Load keywords and credits
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')

# Remove rows with bad IDs.
metadata = metadata.drop([19730, 29503, 35587])

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

In [14]:
metadata.head(2)

adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   

                               homepage    id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story   862  tt0114709                en   
1                                   NaN  8844  tt0113497                en   

  original_title                                           overview  \
0      Toy Story  Led by Woody, Andy's toys live happily in his ...   
1        Jumanji  When siblings Judy and Peter discover an encha...   

                         ...                          \
0                        ...                           
1                        ...                           

                                    spoken_languages    status  \
0           [{'iso_639_1': 'en', 'name': 'English'}]  Released   
1  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...  Released   

                                     tagline      title  video  vote_average  \
0                                        NaN  Toy Story  False           7.7   
1  Roll the dice and unleash the excitement!    Jumanji  False           6.9   

   vote_count                                               cast  \
0      5415.0  [{'cast_id': 14, 'character': 'Woody (voice)',...   
1      2413.0  [{'cast_id': 1, 'character': 'Alan Parrish', '...   

                                                crew  \
0  [{'credit_id': '52fe4284c3a36847f8024f49', 'de...   
1  [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...   

                                            keywords  
0  [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...  
1  [{'id': 10090, 'name': 'board game'}, {'id': 1...  

[2 rows x 27 columns]

In [15]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

In [16]:
import numpy as np
# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
        
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [17]:
# Define new director, cast, genres and keywords features that are in a suitable form.
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [18]:
# Print the new features of the first 3 films
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

title                                            cast  \
0         Toy Story             [Tom Hanks, Tim Allen, Don Rickles]   
1           Jumanji  [Robin Williams, Jonathan Hyde, Kirsten Dunst]   
2  Grumpier Old Men      [Walter Matthau, Jack Lemmon, Ann-Margret]   

        director                                           keywords  \
0  John Lasseter                               [jealousy, toy, boy]   
1   Joe Johnston  [board game, disappearance, based on children'...   
2  Howard Deutch       [fishing, best friend, duringcreditsstinger]   

                         genres  
0   [Animation, Comedy, Family]  
1  [Adventure, Fantasy, Family]  
2             [Romance, Comedy]

In [19]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [20]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

In [21]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)

In [25]:
# it is too hard to proprocessing all the metadata
# Filter out all qualified movies into a new DataFrame
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(4606, 29)

In [30]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

# here have to specify the "decode_error='ignore', other wise the fit_tranform gets encode error.
count = CountVectorizer(stop_words='english',decode_error='ignore')
count_matrix = count.fit_transform(q_movies['soup'])

# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

# Reset index of your main DataFrame and construct reverse mapping as before
q_movies = q_movies.reset_index()
indices = pd.Series(q_movies.index, index=q_movies['title'])

In [31]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

2411            The Dark Knight
1956              Batman Begins
2176               The Prestige
4014    Kidnapping Mr. Heineken
2930                     Faster
3026                     Takers
3102                 The Double
1467          Escape to Victory
1783                   Catwoman
2140                      Chaos
Name: title, dtype: object